In [166]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop\GraphicalPR


In [167]:
import numpy as np
from core.uncertain_array import UncertainArray as UA
from graph.prior import SparsePrior, GaussianPrior
from graph.propagator.unitary_propagator import UnitaryPropagator
from graph.measurement import MaskedGaussianMeasurement, GaussianMeasurement
from core.linalg_utils import random_unitary_matrix
from core.linalg_utils import random_binary_mask

## Gaussian Measurement & Gaussian Prior

In [168]:
n = 128
noise_var = 0.01
np.random.seed(42)
true_x = (np.random.randn(n) + 1j * np.random.randn(n)) * np.sqrt(1/2)
U = random_unitary_matrix(n, seed=1)
true_y = U @ true_x
noise = (np.random.randn(n) + 1j * np.random.randn(n)) * np.sqrt(noise_var/2)
observed = true_y + noise

In [169]:
X = GaussianPrior(shape = (n,))
Y = UnitaryPropagator(U) @ X
Z = GaussianMeasurement(Y, observed, var = noise_var)

In [170]:
for _ in range(5):
    X.parent.forward()
    X.forward()
    Y.parent.forward()
    Y.forward()
    Z.backward()
    Y.backward()
    Y.parent.backward()
    X.backward()
    x_belief = X.compute_belief()
    print("Reconstruction MSE:", np.mean(np.abs(x_belief.data - true_x) ** 2))

Reconstruction MSE: 0.009395119348457421
Reconstruction MSE: 0.009285609456118827
Reconstruction MSE: 0.009285609456118827
Reconstruction MSE: 0.009285609456118827
Reconstruction MSE: 0.009285609456118827


## Sparse Prior & GaussianMeasurement

In [171]:
# 1. パラメータと真のスパース信号
n = 128
sparsity = 0.1
noise_var = 0.01
mask_ratio = 0.5
np.random.seed(42)

true_x = np.zeros(n, dtype=np.complex128)
nonzero_indices = np.random.choice(n, int(n * sparsity), replace=False)
true_x[nonzero_indices] = np.random.randn(len(nonzero_indices)) + 1j * np.random.randn(len(nonzero_indices))

# 2. ユニタリ行列と観測
U = random_unitary_matrix(n, seed=1)
true_y = U @ true_x
noise = (np.random.randn(n) + 1j * np.random.randn(n)) * np.sqrt(noise_var/2)
observed = true_y + noise

In [172]:
# 3. グラフ構築
X = SparsePrior(rho=sparsity, shape=(n,), damping=0.02)
Y = UnitaryPropagator(U) @ X
Z = GaussianMeasurement(Y, observed, var=noise_var)

In [173]:
# 4. 推論ステップ（最小：1往復）
for _ in range(10):
    X.parent.forward()
    X.forward()
    Y.parent.forward()
    Y.forward()
    Z.backward()
    Y.backward()
    Y.parent.backward()
    X.backward()
    x_belief = X.compute_belief()
    print("Reconstruction MSE:", np.mean(np.abs(x_belief.data - true_x) ** 2))

Reconstruction MSE: 0.01125754028023061
Reconstruction MSE: 0.03092081738063377
Reconstruction MSE: 0.004688572059539616
Reconstruction MSE: 0.009060707341559233
Reconstruction MSE: 0.0023533419011834925
Reconstruction MSE: 0.00250700332388568
Reconstruction MSE: 0.013572688527010093
Reconstruction MSE: 0.002492859365482196
Reconstruction MSE: 0.0023817755365728908
Reconstruction MSE: 0.0024487957610066845


## SparsePrior & MaskedGaussianMeasurement

In [177]:
# 1. パラメータと真のスパース信号
n = 512
sparsity = 0.1
noise_var = 0.01
mask_ratio = 0.8
np.random.seed(21)

true_x = np.zeros(n, dtype=np.complex128)
nonzero_indices = np.random.choice(n, int(n * sparsity), replace=False)
true_x[nonzero_indices] = np.random.randn(len(nonzero_indices)) + 1j * np.random.randn(len(nonzero_indices))

# 2. ユニタリ行列と観測
U = random_unitary_matrix(n, seed=1)
true_y = U @ true_x
noise = (np.random.randn(n) + 1j * np.random.randn(n)) * np.sqrt(noise_var/2)
observed = true_y + noise
#mask
mask = random_binary_mask(n, subsampling_rate= mask_ratio, seed=None)

In [180]:
# 3. グラフ構築
X = SparsePrior(rho=sparsity, shape=(n,), damping=0.1)
Y = UnitaryPropagator(U) @ X
Z = MaskedGaussianMeasurement(Y, observed, var=noise_var, mask = mask)

In [181]:
# 4. 推論ステップ（最小：1往復）
for t in range(50):
    X.parent.forward()
    X.forward()
    Y.parent.forward()
    Y.forward()
    Z.backward()
    Y.backward()
    Y.parent.backward()
    X.backward()
    if t % 5 == 0:
        x_belief = X.compute_belief()
        print("t = ", t, "Reconstruction MSE:", np.mean(np.abs(x_belief.data - true_x) ** 2))

t =  0 Reconstruction MSE: 0.24573165885350093
t =  5 Reconstruction MSE: 0.4848598810405721
t =  10 Reconstruction MSE: 0.1961003112547678
t =  15 Reconstruction MSE: 0.2575899458199481
t =  20 Reconstruction MSE: 0.1847180786001826
t =  25 Reconstruction MSE: 1.9455527769448753
t =  30 Reconstruction MSE: 0.17837739810588266
t =  35 Reconstruction MSE: 464.8650111139667
t =  40 Reconstruction MSE: 0.1614783811845183
t =  45 Reconstruction MSE: 0.18688552956213722
